# Basic PUMS Analysis with OpenDP

This notebook is a brief tutorial on doing data analysis within the OpenDP system.

We start out by establishing what is considered public knowledge about the data.
Generally speaking, it is better to wait to load your data until you are ready to invoke the function.
This helps keep us honest while we are writing data preprocessors,
because it is tempting to look at the data to help fill missing parameters.

In [1]:
# establish public information
col_names = ["age", "sex", "educ", "race", "income", "married"]
# the greatest number of records that any one individual can influence in the dataset
max_influence = 1
# we can also reasonably intuit that age and income will be numeric,
#     as well as bounds for them, without looking at the data
age_bounds = (0, 100)
income_bounds = (0, 150_000)

Any time you collect non-DP information from the private data to build your measurement,
the privacy budget will be underestimated.

The vetting process is currently underway for the code in the OpenDP Library.
Any constructors that have not been vetted may still be accessed if you opt-in to "contrib".

In [ ]:
from opendp.mod import enable_features
enable_features('contrib')

### Working with CSV data
Let's examine how we can process csv data with Transformations.

I'm going to pull a few constructors from the [Dataframes section in the user guide](https://docs.opendp.org/en/stable/user/transformation-constructors.html#dataframe).

We start with `make_split_dataframe` to parse one large string containing all the csv data into a dataframe.
`make_split_dataframe` expects us to pass column names, which we can grab out of the public information.
`make_select_column` will then index into the dataframe to pull out a column where the elements have a given type `TOA`.
The `TOA` argument won't cast for you; the casting comes later!

In [2]:
from opendp.trans import make_split_dataframe, make_select_column
income_preprocessor = (
    # Convert data into a dataframe where columns are of type Vec<str>
    make_split_dataframe(separator=",", col_names=col_names) >>
    # Selects a column of df, Vec<str>
    make_select_column(key="income", TOA=str)
)

For the sake of exposition, we're going to go ahead and load up the data.


In [3]:
import os
data_path = os.path.join('.', 'data', 'PUMS_california_demographics_1000', 'data.csv')

with open(data_path) as input_file:
    data = input_file.read()

print('\n'.join(data.split('\n')[:6]))

59,1,9,1,0,1
31,0,1,3,17000,0
36,1,11,1,0,1
54,1,11,1,9100,1
39,0,5,3,37000,0
34,0,9,1,0,1


As we can see from the first few rows, it is intentional that there are no column names in the data.
If your data has column names, you will want to strip them out before passing data into your function.

Now if you run the transformation on the data, you will get a list of incomes as strings.
I've limited the output to just the first few income values.

In [4]:
transformed = income_preprocessor(data)
print(type(transformed))
print(transformed[:6])

<class 'list'>
['0', '17000', '0', '9100', '37000', '0']


### Casting
Income doesn't make sense as a string for our purposes,
so we can just extend the previous preprocessor to also cast and impute.

In [5]:
from opendp.trans import make_cast, make_impute_constant

# make a transformation that casts from a vector of strings to a vector of ints
cast_str_int = (
    # Cast Vec<str> to Vec<Option<int>>
    make_cast(TIA=str, TOA=int) >>
    # Replace any elements that failed to parse with 0, emitting a Vec<int>
    make_impute_constant(0)
)

# replace the previous preprocessor: extend it with the caster
income_preprocessor = income_preprocessor >> cast_str_int
print(income_preprocessor(data)[:6])

[0, 17000, 0, 9100, 37000, 0]


Great! Now we have integer income data from our CSV.
A quick aside, keep in mind that we can invoke transformations on almost anything to do some testing.
For example, we still have a handle on ``cast_str_int``, don't we?

In [6]:
cast_str_int(["null", "1.", "2"])

[0, 0, 2]

### Private Count
Time to compute our first aggregate statistic.
Suppose we want to know the number of records in the dataset.

We can use the [list of aggregators](https://docs.opendp.org/en/stable/user/transformation-constructors.html#aggregators)
in the Transformation Constructors section of the user guide to find `make_count`.

In [7]:
from opendp.trans import make_count
count = income_preprocessor >> make_count(TIA=int)
# NOT a DP release!
count_response = count(data)

Be careful!
`count` is still only a transformation,
so the output in `count_response` is not a differentially private release.
You will need to chain with a measurement to create a differentially private release.

We use `make_base_discrete_laplace` below, because `make_base_discrete_laplace` has an integer support.
Notice that we now import from `opendp.meas`, and the resulting `type(dp_count)` is `Measurement`. This tells us that the output will be a differentially private release.

In [8]:
from opendp.meas import make_base_discrete_laplace
dp_count = count >> make_base_discrete_laplace(scale=1.)


In any realistic situation, you would likely want to estimate the budget utilization before you make a release.
Use a search utility to quantify the privacy expenditure of this release.

In [9]:
from opendp.mod import binary_search

# estimate the budget...
epsilon = binary_search(
    lambda eps: dp_count.check(d_in=max_influence, d_out=eps),
    bounds=(0., 100.))
print("DP count budget:", epsilon)

# ...and then release
count_release = dp_count(data)
print("DP count:", count_release)

DP count budget: 1.0000000009313226
DP count: 1000


### Private Sum

Suppose we want to know the total income of our dataset.
First, take a look at [the list of aggregators](https://docs.opendp.org/en/stable/user/transformation-constructors.html#aggregators).
`make_bounded_sum` seems to meet our requirements.
As indicated by the function's API documentation, it expects bounded data,
so we'll also need to chain the transformation from `make_clamp` with the `income_preprocessor`.

In [10]:
from opendp.trans import make_clamp, make_bounded_sum
bounded_income_sum = (
    income_preprocessor >>
    # Clamp income values
    make_clamp(bounds=income_bounds) >>
    # These bounds must be identical to the clamp bounds, otherwise chaining will fail
    make_bounded_sum(bounds=income_bounds)
)

In this example, instead of just passing a scale into `make_base_discrete_laplace`,
lets say I want whatever scale will make my measurement 1-epsilon DP.
Again, I can use a search utility to find such a scale.

In [11]:
from opendp.mod import binary_search_param

discovered_scale = binary_search_param(
    lambda s: bounded_income_sum >> make_base_discrete_laplace(scale=s),
    d_in=max_influence,
    d_out=1.)

dp_sum = bounded_income_sum >> make_base_discrete_laplace(scale=discovered_scale)


Or more succinctly...

In [12]:
from opendp.mod import binary_search_chain

dp_sum = binary_search_chain(
    lambda s: bounded_income_sum >> make_base_discrete_laplace(scale=s),
    d_in=max_influence,
    d_out=1.)

# ...and make our 1-epsilon DP release
print("DP sum:", dp_sum(data))

DP sum: 30255829


### Private Mean

We may be more interested in the mean age.
Just like before, we reference the docs to find `make_sized_bounded_mean`.
The name of the constructor indicates that it expects sized, bounded data,
and the docstring points us toward preprocessors we can use.

Sized data is data that has a known number of rows.
The constructor enforces this requirement
because knowledge of the dataset size was necessary to establish the privacy proof.

Luckily, we've already made a DP release of the number of rows in the dataset,
which we can reuse as an argument here.

Putting the previous sections together, our bounded mean age is:

In [13]:
from opendp.trans import make_cast_default, make_bounded_resize, make_sized_bounded_mean
from opendp.mod import OpenDPException

try:
    mean_age_preprocessor = (
        # Convert data into a dataframe of string columns
        make_split_dataframe(separator=",", col_names=col_names) >>
        # Selects a column of df, Vec<str>
        make_select_column(key="age", TOA=str) >>
        # Cast the column as Vec<float>, and fill nulls with the default value, 0.
        make_cast_default(TIA=str, TOA=float) >>
        # Clamp age values
        make_clamp(bounds=age_bounds)
    )
except OpenDPException as err:
    assert err.message.startswith("Intermediate domains don't match.")
    print(err.message)

Intermediate domains don't match. See https://github.com/opendp/opendp/discussions/297
    The structure of the intermediate domains are the same, but the types or parameters differ.
    shared_domain: VectorDomain(AllDomain())



Wait a second! The intermediate domains don't match?
In this case, we casted to float-valued data, but `make_clamp` was built with integer-valued bounds,
so the clamp is expecting integer data.
Therefore, the output of the cast is not a valid input to the clamp.
We can fix this by adjusting the bounds and trying again.

In [14]:
float_age_bounds = tuple(map(float, age_bounds))

mean_age_preprocessor = (
    # Convert data into a dataframe of string columns
    make_split_dataframe(separator=",", col_names=col_names) >>
    # Selects a column of df, Vec<str>
    make_select_column(key="age", TOA=str) >>
    # Cast the column as Vec<float>, and fill nulls with the default value, 0.
    make_cast_default(TIA=str, TOA=float) >>
    # Clamp age values
    make_clamp(bounds=float_age_bounds) >>
    # Resize the dataset to length `count_release`.
    #     If there are fewer than `count_release` rows in the data, fill with a constant of 20.
    #     If there are more than `count_release` rows in the data, only keep `count_release` rows
    make_bounded_resize(size=count_release, bounds=float_age_bounds, constant=20.) >>
    # Compute the mean
    make_sized_bounded_mean(size=count_release, bounds=float_age_bounds)
)


I stopped just short of chaining with a measurement because we're working with float data.
There are some extra considerations to take in mind with floating-point data,
which are [covered in the user guide](https://docs.opendp.org/en/latest/user/measurement-constructors.html#floating-point).

With the assumption that you understand the ramifications, I'll go ahead and finish the query.

In [15]:
from opendp.meas import make_base_laplace
from opendp.mod import enable_features
enable_features("floating-point")
# add laplace noise
dp_mean = mean_age_preprocessor >> make_base_laplace(scale=1.0)

mean_release = dp_mean(data)
print("DP mean:", mean_release)

DP mean: 44.303365954161585


Depending on your use-case, you may find greater utility separately releasing a DP sum and a DP count,
and then postprocessing them into the mean.
In the above mean example, you could even take advantage of this to avoid using floating-point numbers.


### Private Variance

This is the last quick example, just to show a complete computation chain.
In this example, I chain with the gaussian mechanism instead, with a budget of .1 epsilon, 1e-5 delta.

In [16]:
from opendp.meas import make_base_gaussian

variance = (
    # Convert data into a dataframe of string columns
    make_split_dataframe(separator=",", col_names=col_names) >>
    # Selects a column of df, Vec<str>
    make_select_column(key="age", TOA=str) >>
    # Cast the column as Vec<float>, and fill nulls with the default value, 0.
    make_cast_default(TIA=str, TOA=float) >>
    # Clamp age values
    make_clamp(bounds=float_age_bounds) >>
    # Resize the dataset to length `count_release`.
    make_bounded_resize(size=count_release, bounds=float_age_bounds, constant=20.) >>
    # Compute the mean
    make_sized_bounded_mean(size=count_release, bounds=float_age_bounds)
)

dp_variance = binary_search_chain(
    lambda s: variance >> make_base_gaussian(scale=s),
    d_in=max_influence,
    d_out=(.1, 1e-5) # (epsilon, delta)-DP
)

print("DP variance:", dp_variance(data))

DP variance: 41.24415287647741
